# MVD 3. cvičení

## 1. část - Stažení a načtení předtrénovaných GloVe slovních reprezentací


### Stažení dat

Pro toto cvičení můžete používat předtrénované Word2Vec nebo GloVe vektory. Následující text se vztahuje ke GloVe vektorům, které byly vybrány z důvodu menší velikosti. 

Základní verzi vektorů lze stáhnout [zde (GloVe link)](https://huggingface.co/stanfordnlp/glove/resolve/main/glove.6B.zip).

Wikipedia 2014 + Gigaword 5 (6B tokens, 400K vocab, uncased, 300d vectors, 822 MB download)

Po rozbalení staženého archivu budete mít několik verzí o různé dimenzi vektorů - 50d, 100d, 200d, 300d. Je doporučeno začít pracovat s nejmenšími vektory a na větších spouštět až závěrečné řešení.

### Načtení dat

Data jsou uložena v textovém souboru, kde je na každém řádku slovo a jeho příslušný vektor.

Načtení je vhodné provést do dvou proměnných -> words, vectors. Words bude list o délce *n* a vectors bude matice o velikosti *(n, d)*. 

Zároveň vytvořte slovník word2idx, pomocí kterého lze získat index libovolného slova (pomocí *word2idx['queen']*).

In [21]:
import numpy as np

In [29]:
# filename = 'glove.6B/glove.6B.50d.txt'
# filename = 'glove.6B/glove.6B.100d.txt'
filename = 'glove.6B/glove.6B.200d.txt'
# filename = 'glove.6B/glove.6B.300d.txt'

with open(filename, 'r') as f:
    words = []
    vectors = []
    word2idx = {}
    for idx, line in enumerate(f):
        line = line.strip().split()
        word = line[0]
        vec = line[1:]
        words.append(word)
        vectors.append(vec)
        word2idx[word] = idx
        
words = np.array(words)
vectors = np.array(vectors, dtype=np.float32)
print(words[:5])
print(vectors[:5])

['the' ',' '.' 'of' 'to']
[[-7.1549e-02  9.3459e-02  2.3738e-02 -9.0339e-02  5.6123e-02  3.2547e-01
  -3.9796e-01 -9.2139e-02  6.1181e-02 -1.8950e-01  1.3061e-01  1.4349e-01
   1.1479e-02  3.8158e-01  5.4030e-01 -1.4088e-01  2.4315e-01  2.3036e-01
  -5.5339e-01  4.8154e-02  4.5662e-01  3.2338e+00  2.0199e-02  4.9019e-02
  -1.4132e-02  7.6017e-02 -1.1527e-01  2.0060e-01 -7.7657e-02  2.4328e-01
   1.6368e-01 -3.4118e-01 -6.6070e-02  1.0152e-01  3.8232e-02 -1.7668e-01
  -8.8153e-01 -3.3895e-01 -3.5481e-02 -5.5095e-01 -1.6899e-02 -4.3982e-01
   3.9004e-02  4.0447e-01 -2.5880e-01  6.4594e-01  2.6641e-01  2.8009e-01
  -2.4625e-02  6.3302e-01 -3.1700e-01  1.0271e-01  3.0886e-01  9.7792e-02
  -3.8227e-01  8.6552e-02  4.7075e-02  2.3511e-01 -3.2127e-01 -2.8538e-01
   1.6670e-01 -4.9707e-03 -6.2714e-01 -2.4904e-01  2.9713e-01  1.4379e-01
  -1.2325e-01 -5.8178e-02 -1.0290e-03 -8.2126e-02  3.6935e-01 -5.8442e-04
   3.4286e-01  2.8426e-01 -6.8599e-02  6.5747e-01 -2.9087e-02  1.6184e-01
   7.3672e-0

## 2. část - Kosinová podobnost

Vytvořte funkci cossim, která bude vracet kosinovu podobnost dvou vstupních vektorů.

<br>
<center>
$
similarity(a,b) = cos(\theta) = \frac{a \cdot b}{\lVert a \lVert \lVert b \lVert}
$
</center>

In [26]:
def cossim(a, b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

## 3. část - Slovní analogie

Nejznámější slovní analogií je z Word2Vec $f("king") - f("man") = f("queen") - f("woman")$

1. Vytvořte skript pro hledání analogií $f("king") - f("man") = f("??") - f("woman")$ a vyzkoušejte i nějaké další.
2. Vypište 5 nejpodobnějších slov

In [30]:
def find_analogy(word1, word2, word3, words, vectors, word2idx, top=5):
    idx1 = word2idx[word1]
    idx2 = word2idx[word2]
    idx3 = word2idx[word3]
    
    vec1 = vectors[idx1]
    vec2 = vectors[idx2]
    vec3 = vectors[idx3]
    
    unknown_vec = vec2 - vec1 + vec3
    
    similarities = np.array([cossim(unknown_vec, v) for v in vectors])
    
    # Exclude the input words from the results
    exclude_indices = {idx1, idx2, idx3}
    top_indices = [i for i in np.argsort(similarities)[::-1] if i not in exclude_indices][:top]
    
    return words[top_indices]

In [32]:
test_words = (
    ('king', 'man', 'woman'),
    ('city', 'country', 'river'),
    ('cat', 'kitten', 'dog'),
    ('paris', 'france', 'berlin'),
    ('japan', 'sushi', 'italy'),
)
for ws in test_words:
    result = find_analogy(*ws, words, vectors, word2idx)
    print("{:<{width}}  {}".format(str(ws), result, width=40))



('king', 'man', 'woman')                  ['girl' 'person' 'teenager' 'boy' 'she']
('city', 'country', 'river')              ['rivers' 'tributary' 'flows' 'tributaries' 'confluence']
('cat', 'kitten', 'dog')                  ['puppy' 'puppies' 'rottweiler' 'kittens' 'dogs']
('paris', 'france', 'berlin')             ['germany' 'german' 'austria' 'poland' 'germans']
('japan', 'sushi', 'italy')               ['pasta' 'tuscany' 'italian' 'trattoria' 'ristorante']


### Bonus - Vytvořte vizualizaci slovních analogií

Pro získání bonusového bodu je potřeba vytvořit vizualizaci slovních analogií (redukce dimenze + vizualizace).